In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie_title": "The Lion King",
        "answer": "'The Lion King' is directed by Roger Allers and Rob Minkoff, the main cast is Matthew Broderick (Simba), James Earl Jones (Mufasa), and Budget was $45 million. The box office revenue was $968.5 million, and its genre was Animation, adventure, drama, family, musical. Its synopsis is a coming-of-age story about a young lion named Simba, who, after the tragic death of his father, must learn to embrace his destiny as the rightful king of the Pride Lands, overcoming betrayal and self-doubt along the way.",
    },
    {
        "movie_title": "Pirates of the Caribbean",
        "answer": "'Pirates of the Caribbean' has director/main cast/budget/box office revunue, genre information (like Lion King). Its synopsis is a swashbuckling adventure that follows the eccentric pirate Captain Jack Sparrow as he navigates curses, treasure hunts, and battles with the British Navy, all while trying to outwit both allies and enemies.",
    }
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "By this movie title {movie_title}, could you respond me information about the movie such as the director, the main cast, the budget, the box office revenue, the genre of the movie and a small synopsis?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Response should be including Director, Main Cast, Budget, Box Office Revenue, Genre, and Synopsis."),
        example_prompt,
        ("human", "By this movie title {movie_title}, could you provide me with  information about the movie such as the director, the main cast, the budget, the box office revenue, the genre of the movie and a small synopsis?"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def get_last_movie_title():
    history = memory.load_memory_variables({})
    if history:
        last_interaction = history[-1]
        movie_title = last_interaction.get('input', 'No movie title in memory.')
        return movie_title
    else:
        return "No memory of any movie titles."

def query_memory_for_movie():
    last_movie_title = get_last_movie_title()
    print(f"The last movie title asked about was: {last_movie_title}")



chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie_title):
    result = chain.invoke({"movie_title": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    print(result.content)